# Create a Deployment Manifest

In [ ]:
import sys
sys.path.append('../../../common')
from env_variables import *

In [ ]:
# Manifest filenames
templateManifestFileName = "../../../common/deployment.common.template.json"
deploymentManifestFileName = "../../../common/deployment.lva_yolov3_icpu_gprc.template.json"

## Update Deployment Manifest Template

The following cell will create a custom template based on this sample. It will copy the sample deployment manifest template and add a few more parameters to a new manifest template. For this sample, we are specifying inter communication port properties as shareable between lvaEdge and lvaExtension modules on the IoT Edge device.

In [ ]:
import json

with open(templateManifestFileName) as f:
    data = json.load(f)

# Set lvaEdge container attribute so its shared mem area is accessible by other containers
hostConfigNode = data["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["lvaEdge"]["settings"]["createOptions"]["HostConfig"]
hostConfigNode["IpcMode"] = "shareable"

# Set LVA Extension container attributes to be able to access lvaEdge container's shared mem area
hostConfigNode = data["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["lvaExtension"]["settings"]["createOptions"]["HostConfig"]
hostConfigNode["IpcMode"] = "container:lvaEdge"

# Specify debug output folder on the IoTEdgeDevice
hostConfigNode["Binds"] = ["{}:/lvaextensiondebug".format(debugOutputFolder)]

# Set environment variables for enabling the custom debugging feature and specify debug output folder
lvaExtensionNode = data["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["lvaExtension"]
envVar = {
            "DEBUG" : {
                "value" : "True"
            }
         }

lvaExtensionNode["env"] = envVar

with open(deploymentManifestFileName, "w") as f:
    json.dump(data, f, indent=4)

## Deployment Manifest File
For deploying IoT Edge Modules on to an IoT Edge device, you need a manifest file which is a Json file. A manifest file specifies the details of the modules (IoT Edge Runtime specific Docker container images) that will be deployed into the Edge device. We will use the [deployment.common.template.json](deployment.common.template.json) manifest file as a template for module deployment and update its content specific to this sample. The manifest is organized like the following image:  

<img src="../../../../../../images/_deployment_modules_0.png" width=300px/>  

As shown in the image above, branch 2 of the Json file defines the system modules to be deployed onto the Edge device. These two system modules (edgeAgent, edgeHub) are necessary modules in each IoT Edge device. In branch 3 of the Json file, we define the custom modules that we want to deploy to the edge device.  
The three custom modules in branch 3 of the Json file are as follows: 
* lvaEdge - node where we define the lvaEdge-specific details, such as from which Container Registry address it should be downloaded. (You may use other reference names, but the name must be consistent in other locations within the manifest file or in any other related code.)
* lvaExtension - the reference name we gave for our inference server module that we developed and uploaded into ACR.
* rtspsim - the reference name given to another container image in ACR. This module simulates an IP camera by playing a user-defined video file as if it were being streamed from an IP camera. This module is optional and placed here to aid users who don't have an IP camera but still want to test LVA.

For more details about IoT Edge modules, refer to [Azure IoT Edge documentation](https://docs.microsoft.com/en-us/azure/iot-edge/about-iot-edge).

If you check the contents of the template manifest file, you will see placeholders that start with the character $. 

| **Placeholder** | **Description** | 
| --- | --- | 
| CONTAINER_REGISTRY_USERNAME_myacr | The username that Azure Container Registry (ACR) will use to authenticate with the registry.<br>ACR is a registry of Docker and Open Container Initiative (OCI) images.| 
| CONTAINER_REGISTRY_PASSWORD_myacr | The password that Azure Container Registry (ACR) will use to authenticate with the registry.<br>When run, a Active Directory token is created to authenticate your session with the container registry.| 
| CONTAINER_IMAGE_NAME | The name of the container image that will be used for deployment.<br>Once uploaded to ACR, it can be called through CONTAINER_REGISTRY_USERNAME_myacr.azurecr.io/CONTAINER_IMAGE_NAME. |
| SUBSCRIPTION_ID | The ID associated with your Azure subscription, typically containing dashes within the string.<br>A subscription represents a grouping of Azure resources. An invoice is generated at the subscription scope. | 
| RESOURCE_GROUP | The name of the resource group that holds your resources.<br>A resource group is a container that holds related resources for an Azure solution. | 
| AMS_ACCOUNT | The name of your Azure Media Services (AMS) resource.<br>AMS is a cloud-based media workflow platform to index, package, protect, and stream video. |
| AAD_TENANT_ID | The Azure Active Directory (AAD) tenant ID, which is used as a reference to a directory.<br>An AAD directory object holds security-related objects such as user accounts, applications, and groups.| 
| AAD_SERVICE_PRINCIPAL_ID | The ID associated with the AAD's service principal.<br>A service principal is a security identity used by user-created apps, services, and tools to access Azure resources. | 
| AAD_SERVICE_PRINCIPAL_SECRET | A secret string associated with the AAD's service principal.<br>The AAD service principal secret adds a layer of security at the service principal scope. | 
| INPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the desired input video is stored.<br>This path must be specified in order for LVA to find the correct videos to process. | 
| OUTPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the processed video output will be stored.<br>After processing is complete, the processed video will be sent and stored in this folder. | 

In the [previous section](../../../common/upload_container_image_to_acr.ipynb), we pushed our inference server container image to the cloud using Azure Container Registry (ACR). In doing so, ACR created a .env file containing the credentials to your container registry. These credentials are shared with your IoT Edge device so that it has access to pull the container image.

We will use the credentials found in the .env file to create an IoT Edge device module deployment manifest file in Json format. The credentials found in the .env file will replace the placeholders we set in our template [deployment.common.template.json](deployment.common.template.json) manifest file.

How does this work? The deployment manifest file is a file that defines exactly which modules you want deployed on a device, how they should be configured, and how they can communicate with each other and the Cloud. When you transform the template into a true deployment manifest, the placeholders in the table above are replaced with values taken from other solution files. Credentials placed in this file are then shared with the IoT Edge runtime to pull your container images onto the IoT Edge device. This configuration is for security purposes, as the .env file is git ignored but the deployment template is not. You should double check by opening the .env file and confirming that your credentials are included. If not, add them manually by locating the appropriate services on Azure and save the .env file.